In [1]:
import pandas as pd

/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import chardet
chardet.detect(open('ultimate_data_challenge.json','rb').read())

{'confidence': 1.0, 'encoding': 'ascii'}

In [3]:
file=open('ultimate_data_challenge.json', 'r')
file_content=file.read()
file.close()

In [4]:
import json
data=json.loads(file_content)
df_data=pd.DataFrame(data)

In [5]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null object
phone                     49604 non-null object
signup_date               50000 non-null object
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
ultimate_black_user       50000 non-null bool
weekday_pct               50000 non-null float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [6]:
df_data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,ultimate_black_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [7]:
df_data.city.unique()

array(["King's Landing", 'Astapor', 'Winterfell'], dtype=object)

In [8]:
df_data['last_trip_date'] = pd.to_datetime(df_data['last_trip_date'])
df_data['signup_date'] = pd.to_datetime(df_data['signup_date'])

In [9]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null datetime64[ns]
phone                     49604 non-null object
signup_date               50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
ultimate_black_user       50000 non-null bool
weekday_pct               50000 non-null float64
dtypes: bool(1), datetime64[ns](2), float64(6), int64(1), object(2)
memory usage: 4.2+ MB


In [10]:
latest_date = max(df_data['last_trip_date'])

In [11]:
active_day = latest_date - df_data['last_trip_date']

In [12]:
active_day_ser  = active_day.dt.days.astype(int)

In [13]:
retain_user = active_day_ser.map(lambda x: 1 if x <= 30 else 0)

In [14]:
retain_user.value_counts()

0    31196
1    18804
Name: last_trip_date, dtype: int64

In [15]:
pct_retain = retain_user.value_counts()[1]*100/retain_user.count()
print ('{}% of riders were retained at the end of the survey period'.format(pct_retain))

37.608% of riders were retained at the end of the survey period


In [16]:
active_month = df_data['last_trip_date'] - df_data['signup_date']

In [17]:
import numpy as np
active_month_ser = active_month/np.timedelta64(1,"M")

In [18]:
active_month_ser = active_month_ser.astype(int)

In [19]:
active_user = active_month_ser.map(lambda x: 1 if x >= 5.0 else 0)

In [20]:
active_user.value_counts()

0    38806
1    11194
dtype: int64

In [21]:
df_data['avg_rating_by_driver'].fillna(df_data['avg_rating_by_driver'].mean(), inplace=True)
df_data['avg_rating_of_driver'].fillna(df_data['avg_rating_of_driver'].mean(), inplace=True)

In [22]:
df_data.phone.value_counts()

iPhone     34582
Android    15022
Name: phone, dtype: int64

In [23]:
df_data['phone'].fillna('iPhone',inplace=True)

In [24]:
city = pd.get_dummies(df_data['city'])
phone = pd.get_dummies(df_data['phone'])
black = pd.get_dummies(df_data['ultimate_black_user'],prefix='black')

In [25]:
df_train = df_data.drop(['last_trip_date','signup_date','surge_pct','city','phone','ultimate_black_user'], axis=1)

In [26]:
df_train = df_train.join([city,phone,black], how='left')

In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      50000 non-null float64
avg_rating_of_driver      50000 non-null float64
avg_surge                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
weekday_pct               50000 non-null float64
Astapor                   50000 non-null uint8
King's Landing            50000 non-null uint8
Winterfell                50000 non-null uint8
Android                   50000 non-null uint8
iPhone                    50000 non-null uint8
black_False               50000 non-null uint8
black_True                50000 non-null uint8
dtypes: float64(5), int64(1), uint8(7)
memory usage: 2.6 MB


In [28]:
df_test = active_user.to_frame()

In [29]:
df_test.columns = ['active']

In [30]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 1 columns):
active    50000 non-null int64
dtypes: int64(1)
memory usage: 390.7 KB


In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Anaconda/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [32]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(df_train)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, df_test, test_size=0.33, random_state=42)

In [34]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [35]:
y_pred = clf.predict(X_test)

In [36]:
print('=====Feature importance=======')
for idx, val in enumerate(clf.feature_importances_):
    print("{:30}{:3f}".format(df_train.columns[idx], val))
print('-----------------------------------------------')

print("{:30}{:3f}".format('F1 score', f1_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test accuracy', precision_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test recall', recall_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test precision', accuracy_score(y_test, y_pred)))
print()

print('====Confusion Matrix=====')
print(confusion_matrix(y_test, y_pred))

=====Feature importance=======
avg_dist                      0.332744
avg_rating_by_driver          0.134234
avg_rating_of_driver          0.095215
avg_surge                     0.109257
trips_in_first_30_days        0.077484
weekday_pct                   0.155530
Astapor                       0.013142
King's Landing                0.025362
Winterfell                    0.008484
Android                       0.017623
iPhone                        0.007912
black_False                   0.012960
black_True                    0.010052
-----------------------------------------------
F1 score                      0.402155
Test accuracy                 0.397392
Test recall                   0.407034
Test precision                0.730970

====Confusion Matrix=====
[[10568  2264]
 [ 2175  1493]]


In [37]:
new_train = df_train.drop(['trips_in_first_30_days','Astapor',"King's Landing",'Winterfell','Android','iPhone','black_True','black_False'], axis=1)

In [38]:
scaler = StandardScaler()
scaled_new_X = scaler.fit_transform(new_train)
X_train, X_test, y_train, y_test = train_test_split(scaled_new_X, df_test, test_size=0.33, random_state=42)
clf1 = DecisionTreeClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)

In [39]:
print('=====Feature importance=======')
for idx, val in enumerate(clf1.feature_importances_):
    print("{:30}{:3f}".format(new_train.columns[idx], val))
print('-----------------------------------------------')

print("{:30}{:3f}".format('F1 score', f1_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test accuracy', precision_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test recall', recall_score(y_test, y_pred)))
print("{:30}{:3f}".format('Test precision', accuracy_score(y_test, y_pred)))
print()

print('====Confusion Matrix=====')
print(confusion_matrix(y_test, y_pred))

=====Feature importance=======
avg_dist                      0.430336
avg_rating_by_driver          0.138155
avg_rating_of_driver          0.110042
avg_surge                     0.134223
weekday_pct                   0.187244
-----------------------------------------------
F1 score                      0.361068
Test accuracy                 0.370657
Test recall                   0.351963
Test precision                0.723091

====Confusion Matrix=====
[[10640  2192]
 [ 2377  1291]]


The model is a preliminiary one.Hence, the metrics of model, specially the recall seem poor. However, the model is useful in 
identifying importnat features in predicting rider retention.

The model shows that the average distnace traveled by the rider in first 30 days,  and the rating provided by rider as well as driver
is important in retaining the rider. The percentage of travel during weekdays also is important in retaining rider. Hence,
Early impression on the rider seems to be important. Also, ultimate should focus more on weekday riders as they seem to
be the active for longer than the ones who travel on weekends.